In [1]:
# Libraries import
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import os
from os import listdir
from os.path import isfile, join

%matplotlib inline

# Get current/root directory
root = os.getcwd()

In [2]:
# Plot style
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18, 8)

In [3]:
data = []
sig_arr = []
pid = 1
N = 256

# compute--
# magnitude   phase   signature   distance   sector

In [4]:
# Compute phase angle given r,c
def compute_phase(r, c):
    x = r - (N / 2)
    y = (N / 2) - c
    q = 0 if x == 0 or y == 0 else (y / x)
    phase = math.atan(q)
    return phase


# Compute distance given r,c
def compute_distance(r, c):
    x = r - (N / 2)
    y = (N / 2) - c
    d = math.sqrt(x*x+y*y)
    return d


# Compute distance given r,c
def compute_sector(phase):
    phase_d = abs(np.degrees(phase))
    #print(phase_d)
    v = (phase_d/22.5)//1 #get only integer
    if v == 16:
        return 0
    return v


# Computing slop
def compute_theta(p1, p2, p3):
    # p1(x1,y1)
    # p2(x2,y2)
    # p3(x3,y3)
    HX1 = []
    HX2 = []
    
    HX1.append(p1[0]-5) # x1 0
    HX1.append(p1[1]) # y1 1
    HX1.append(p1[0]+5) # x2 2
    HX1.append(p1[1]) # y2 3
    
    HX2.append(p2[0]-5)
    HX2.append(p2[1])
    HX2.append(p2[0]+5)
    HX2.append(p2[1])
    
    M1 = (HX1[3]-HX1[1])/(HX1[2]-HX1[0]) # Slope of HX1
    M2 = (p2[1]-p1[1])/(p2[0]-p1[0]) # Slope of p1p2
    M3 = (HX2[3]-HX2[1])/(HX2[2]-HX2[0]) # Slope of HX2
    M4 = (p3[1]-p2[1])/(p3[0]-p2[0]) # Slope of p2p3
    
    TAN1 = abs((M2-M1)/(1+M1*M2))
    THETA1 = math.atan(TAN1)
    
    TAN2 = abs((M4-M3)/(1+M3*M4))
    THETA2 = math.atan(TAN2)
    
    # For negative slope, THETA1 is negative
    if M2 < 0:
        THETA1 = -THETA1
        
    if p3[1] < p2[1]:
        THETA2 = -THETA2
    
    THETA3 = THETA2-THETA1
    
    #OBJ.append(THETA3)
    
    return THETA3

In [5]:
loc = 'D:/SCA/256_256/txt'
files = [f for f in listdir(loc) if isfile(join(loc, f))]

for file in files:
    print(file)
    
    arr = file.split('_')
    sl_no = arr[0]
    slice_no = arr[1]
    subslice_no= arr[2]
    otype = arr[3]
    x_0 = arr[4]
    y_0 = arr[4].split('.')[0]
    
    dataset=pd.read_csv(loc + "/" + file, header=None, delimiter="\t")   
    dataset = dataset.values
    
    for ds in dataset:
        temp = []
        st = ds[0].replace('<', '')
        st = st.replace('>', '')
        d = st.split(',')
        d = np.array(d, dtype=np.float32)

        spectrum = d[2]
        phase = compute_phase(d[0], d[1])
        distance = compute_distance(d[0], d[1])
        sector = compute_sector(phase)
        signature_angle = 0
        
        if len(sig_arr) < 3:
            p = []
            x = d[0]-(N / 2) #r - (N / 2)
            y = (N / 2) - d[1] #(N / 2) - c
            p.append(x)
            p.append(y)
            sig_arr.append(p)
            
            if len(sig_arr) == 3:
                THETA = compute_theta(sig_arr[0], sig_arr[1], sig_arr[2])
                signature_angle = THETA
            else:
                signature_angle = 0
        else:
            p = []
            x = d[0]-(N / 2) #r - (N / 2)
            y = (N / 2) - d[1] #(N / 2) - c
            
            p.append(x)
            p.append(y)
            sig_arr[0] = sig_arr[1]
            sig_arr[1] = sig_arr[2]
            sig_arr[2] = p
            
            THETA = compute_theta(sig_arr[0], sig_arr[1], sig_arr[2])
            signature_angle = THETA
        
        if math.isnan(spectrum):
            spectrum = 0
            
        if math.isnan(phase):
            phase = 0
            
        if math.isnan(sector):
            sector = 0
            
        if math.isnan(distance):
            distance = 0
        
        if math.isnan(signature_angle):
            signature_angle = 0
            
        # mag,phase,signature,sector,distance
        str_data = str(spectrum)+','+str(phase)+','+str(signature_angle)+','+str(sector)+','+str(distance)
        
        file_object = open('D:/SCA/256_256/feature_map/test_unet/'+file, 'a+')
        file_object.write(str_data+'\n')
        file_object.close()

102_114_1_c_425_228.txt
102_114_1_c_h_425_228.txt

C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\gc\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars



102_114_1_e_261_153.txt
102_114_1_e_h_261_153.txt
102_115_1_c_248_226.txt
102_115_1_c_271_244.txt
102_115_1_c_275_188.txt
102_115_1_c_293_214.txt
102_115_1_c_295_173.txt
102_115_1_c_313_145.txt
102_115_1_c_319_248.txt
102_115_1_c_396_151.txt
102_115_1_c_419_258.txt
102_115_1_c_423_185.txt
102_115_1_c_425_228.txt
102_115_1_c_h_248_226.txt
102_115_1_e_382_256.txt
102_116_1_c_230_226.txt
102_116_1_c_252_225.txt
102_116_1_c_272_244.txt
102_116_1_c_293_216.txt
102_116_1_c_295_173.txt
102_116_1_c_313_145.txt
102_116_1_c_319_248.txt
102_116_1_c_381_256.txt
102_116_1_c_396_151.txt
102_116_1_c_419_258.txt
102_116_1_c_422_185.txt
102_116_1_c_425_228.txt
102_117_1_c_250_225.txt
102_117_1_c_272_245.txt
102_117_1_c_293_216.txt
102_117_1_c_296_173.txt
102_117_1_c_314_145.txt
102_117_1_c_319_248.txt
102_117_1_c_381_256.txt
102_117_1_c_396_151.txt
102_117_1_c_421_259.txt
102_117_1_c_423_185.txt
102_117_1_c_427_229.txt
102_118_1_c_248_225.txt
102_118_1_c_273_244.txt
102_118_1_c_293_216.txt
102_118_1_c

102_24_1_c_439_217.txt
102_24_1_e_315_110.txt
102_24_1_e_414_181.txt
102_24_1_e_424_252.txt
102_25_1_c_260_249.txt
102_25_1_c_275_214.txt
102_25_1_c_306_255.txt
102_25_1_c_387_253.txt
102_25_1_c_402_141.txt
102_25_1_c_424_251.txt
102_25_1_e_416_181.txt
102_26_1_c_260_249.txt
102_26_1_c_275_214.txt
102_26_1_c_306_255.txt
102_26_1_c_387_253.txt
102_26_1_c_401_141.txt
102_26_1_c_423_251.txt
102_26_1_e_329_235.txt
102_26_1_e_378_225.txt
102_26_1_e_417_181.txt
102_27_1_c_259_249.txt
102_27_1_c_272_216.txt
102_27_1_c_305_255.txt
102_27_1_c_387_253.txt
102_27_1_c_403_142.txt
102_27_1_c_423_181.txt
102_27_1_c_424_252.txt
102_28_1_c_258_250.txt
102_28_1_c_272_216.txt
102_28_1_c_305_255.txt
102_28_1_c_388_253.txt
102_28_1_c_403_142.txt
102_28_1_c_422_181.txt
102_28_1_c_424_252.txt
102_29_1_c_258_251.txt
102_29_1_c_273_216.txt
102_29_1_c_304_255.txt
102_29_1_c_388_253.txt
102_29_1_c_402_141.txt
102_29_1_c_424_252.txt
102_29_1_e_419_181.txt
102_30_1_c_258_250.txt
102_30_1_c_273_216.txt
102_30_1_c_

102_60_1_c_396_144.txt
102_60_1_c_418_250.txt
102_60_1_c_427_183.txt
102_60_1_c_435_221.txt
102_60_1_e_261_156.txt
102_61_1_c_265_245.txt
102_61_1_c_284_213.txt
102_61_1_c_312_251.txt
102_61_1_c_322_135.txt
102_61_1_c_386_253.txt
102_61_1_c_396_144.txt
102_61_1_c_419_250.txt
102_61_1_c_427_183.txt
102_61_1_c_434_220.txt
102_62_1_c_261_248.txt
102_62_1_c_287_212.txt
102_62_1_c_322_135.txt
102_62_1_c_384_253.txt
102_62_1_c_395_144.txt
102_62_1_c_419_250.txt
102_62_1_c_428_185.txt
102_62_1_c_435_221.txt
102_62_1_e_291_172.txt
102_62_1_e_364_211.txt
102_63_1_c_263_247.txt
102_63_1_c_285_212.txt
102_63_1_c_289_173.txt
102_63_1_c_322_135.txt
102_63_1_c_383_253.txt
102_63_1_c_396_144.txt
102_63_1_c_421_251.txt
102_63_1_c_428_185.txt
102_63_1_e_363_210.txt
102_64_1_c_261_248.txt
102_64_1_c_286_212.txt
102_64_1_c_289_172.txt
102_64_1_c_321_135.txt
102_64_1_c_396_144.txt
102_64_1_c_419_250.txt
102_64_1_c_429_185.txt
102_64_1_e_361_210.txt
102_65_1_c_261_248.txt
102_65_1_c_286_179.txt
102_65_1_c_

...

9_107_1_c_377_147.txt
9_107_1_c_413_175.txt
9_107_1_c_422_240.txt
9_107_1_e_219_224.txt
9_107_1_e_329_90.txt
9_107_1_e_390_216.txt
9_107_1_e_401_260.txt
9_107_1_e_460_195.txt
9_108_1_c_257_216.txt
9_108_1_c_264_244.txt
9_108_1_c_275_174.txt
9_108_1_c_295_255.txt
9_108_1_c_298_212.txt
9_108_1_c_317_233.txt
9_108_1_c_319_147.txt
9_108_1_c_369_241.txt
9_108_1_c_377_147.txt
9_108_1_c_391_216.txt
9_108_1_c_414_174.txt
9_108_1_c_423_240.txt
9_108_1_e_401_260.txt
9_108_1_e_459_195.txt
9_109_1_c_257_216.txt
9_109_1_c_264_245.txt
9_109_1_c_275_174.txt
9_109_1_c_294_255.txt
9_109_1_c_298_212.txt
9_109_1_c_318_147.txt
9_109_1_c_319_233.txt
9_109_1_c_369_241.txt
9_109_1_c_376_147.txt
9_109_1_c_391_216.txt
9_109_1_c_400_259.txt
9_109_1_c_415_174.txt
9_109_1_c_423_240.txt
9_109_1_e_459_195.txt
9_10_1_c_239_191.txt
9_10_1_c_265_181.txt
9_10_1_c_294_171.txt
9_10_1_e_276_233.txt
9_10_1_e_373_179.txt
9_10_1_e_453_274.txt
9_110_1_c_256_216.txt
9_110_1_c_263_245.txt
9_110_1_c_275_173.txt
9_110_1_c_294_255

9_131_1_c_319_233.txt
9_131_1_c_369_241.txt
9_131_1_c_397_261.txt
9_131_1_c_415_237.txt
9_131_1_e_212_189.txt
9_131_1_e_212_226.txt
9_131_1_e_216_103.txt
9_131_1_e_217_158.txt
9_131_1_e_239_40.txt
9_131_1_e_246_199.txt
9_131_1_e_248_261.txt
9_131_1_e_258_139.txt
9_131_1_e_258_86.txt
9_131_1_e_279_133.txt
9_131_1_e_296_179.txt
9_131_1_e_296_78.txt
9_131_1_e_429_85.txt
9_131_1_e_434_139.txt
9_131_1_e_438_197.txt
9_131_1_e_477_49.txt
9_131_1_e_478_104.txt
9_13_1_c_268_180.txt
9_13_1_c_426_252.txt
9_13_1_e_288_235.txt
9_13_1_e_322_171.txt
9_13_1_e_381_248.txt
9_13_1_e_398_190.txt
9_14_1_c_271_178.txt
9_14_1_c_435_221.txt
9_14_1_e_323_171.txt
9_14_1_e_339_194.txt
9_14_1_e_381_248.txt
9_14_1_e_402_193.txt
9_14_1_e_450_268.txt
9_15_1_c_272_178.txt
9_15_1_c_436_222.txt
9_15_1_e_382_248.txt
9_15_1_e_398_191.txt
9_15_1_e_438_258.txt
9_16_1_c_272_176.txt
9_16_1_c_306_145.txt
9_16_1_c_437_222.txt
9_16_1_e_292_232.txt
9_16_1_e_381_248.txt
9_16_1_e_437_258.txt
9_17_1_c_306_147.txt
9_17_1_c_459_214.t

9_45_1_c_363_238.txt
9_45_1_c_374_216.txt
9_45_1_c_408_173.txt
9_45_1_c_413_255.txt
9_45_1_c_432_218.txt
9_45_1_c_438_248.txt
9_46_1_c_248_256.txt
9_46_1_c_249_224.txt
9_46_1_c_268_182.txt
9_46_1_c_286_257.txt
9_46_1_c_299_211.txt
9_46_1_c_307_148.txt
9_46_1_c_315_235.txt
9_46_1_c_364_238.txt
9_46_1_c_370_149.txt
9_46_1_c_374_216.txt
9_46_1_c_407_173.txt
9_46_1_c_413_255.txt
9_46_1_c_430_218.txt
9_46_1_c_438_248.txt
9_46_1_e_455_105.txt
9_47_1_c_248_256.txt
9_47_1_c_250_224.txt
9_47_1_c_268_181.txt
9_47_1_c_287_257.txt
9_47_1_c_299_211.txt
9_47_1_c_307_148.txt
9_47_1_c_315_235.txt
9_47_1_c_364_238.txt
9_47_1_c_370_142.txt
9_47_1_c_374_216.txt
9_47_1_c_405_173.txt
9_47_1_c_411_253.txt
9_47_1_c_429_218.txt
9_47_1_c_437_247.txt
9_47_1_e_316_118.txt
9_47_1_e_452_105.txt
9_48_1_c_248_256.txt
9_48_1_c_249_224.txt
9_48_1_c_287_257.txt
9_48_1_c_299_212.txt
9_48_1_c_306_148.txt
9_48_1_c_315_235.txt
9_48_1_c_364_238.txt
9_48_1_c_370_142.txt
9_48_1_c_375_216.txt
9_48_1_c_407_174.txt
9_48_1_c_411_

9_71_1_c_373_147.txt
9_71_1_c_383_217.txt
9_71_1_c_408_257.txt
9_71_1_c_409_173.txt
9_71_1_c_430_243.txt
9_71_1_c_436_220.txt
9_72_1_c_254_249.txt
9_72_1_c_260_217.txt
9_72_1_c_273_178.txt
9_72_1_c_291_257.txt
9_72_1_c_298_211.txt
9_72_1_c_313_147.txt
9_72_1_c_317_234.txt
9_72_1_c_368_240.txt
9_72_1_c_383_217.txt
9_72_1_c_407_256.txt
9_72_1_c_408_172.txt
9_72_1_c_431_244.txt
9_72_1_c_436_220.txt
9_72_1_e_372_145.txt
9_73_1_c_253_249.txt
9_73_1_c_260_217.txt
9_73_1_c_274_178.txt
9_73_1_c_289_256.txt
9_73_1_c_299_211.txt
9_73_1_c_313_148.txt
9_73_1_c_316_234.txt
9_73_1_c_367_240.txt
9_73_1_c_384_217.txt
9_73_1_c_407_257.txt
9_73_1_c_408_172.txt
9_73_1_c_430_243.txt
9_73_1_c_436_220.txt
9_73_1_e_372_145.txt
9_74_1_c_253_250.txt
9_74_1_c_259_217.txt
9_74_1_c_273_178.txt
9_74_1_c_289_257.txt
9_74_1_c_299_211.txt
9_74_1_c_313_147.txt
9_74_1_c_317_234.txt
9_74_1_c_368_240.txt
9_74_1_c_372_147.txt
9_74_1_c_384_217.txt
9_74_1_c_408_173.txt
9_74_1_c_429_243.txt
9_74_1_c_435_220.txt
9_75_1_c_254_

9_98_1_c_369_241.txt
9_98_1_c_377_147.txt
9_98_1_c_394_218.txt
9_98_1_c_403_259.txt
9_98_1_c_415_176.txt
9_98_1_c_425_241.txt
9_98_1_e_257_145.txt
9_98_1_e_466_198.txt
9_98_1_e_482_245.txt
9_99_1_c_255_217.txt
9_99_1_c_260_247.txt
9_99_1_c_277_175.txt
9_99_1_c_292_257.txt
9_99_1_c_298_212.txt
9_99_1_c_315_234.txt
9_99_1_c_318_147.txt
9_99_1_c_370_241.txt
9_99_1_c_377_147.txt
9_99_1_c_393_218.txt
9_99_1_c_402_258.txt
9_99_1_c_416_175.txt
9_99_1_c_425_241.txt
9_99_1_e_259_144.txt
9_99_1_e_465_196.txt
